<a href="https://colab.research.google.com/github/lucabem/CompeticionMineria/blob/main/Presentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cargar paquetes

In [87]:

!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow_addons

import os
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing import image
import numpy as np
import pandas as pd
import tensorflow_text
from google.colab import files
import tensorflow_addons as tfa
import pathlib



Requirement already up-to-date: tensorflow_hub in /usr/local/lib/python3.7/dist-packages (0.11.0)
Requirement already up-to-date: tensorflow_addons in /usr/local/lib/python3.7/dist-packages (0.12.1)


# 2. Cargar modelo

Se carga manualmente.

In [84]:
path_to_model = "/content/model"
model = tf.keras.models.load_model(path_to_model, compile=True, custom_objects={'f1':tfa.metrics.F1Score(num_classes=25, average='weighted')})


In [85]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 25)                2

'\nmodel.compile(loss="categorical_crossentropy",\n              optimizer=opt,\n\t            metrics=[tfa.metrics.F1Score(num_classes=25, average=\'weighted\')],\n              )\n'

## Cargar el Dataset

In [88]:
!rm -rf /root/.keras/datasets/*
dataset_url = 'https://github.com/lucabem/CompeticionMineria/blob/main/data/dataset_images.zip?raw=true'
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='train_data', 
                                   extract=True)

!rm -rf /root/.keras/datasets/train_data.tar.gz /root/.keras/datasets/train_data
!ls -l /root/.keras/datasets/
train_path = pathlib.Path('/root/.keras/datasets/ImagesTrain')
image_count = len(list(train_path.glob('*/*.jpg')))
print(image_count)

78069760/78067648 [==============================] - 1s 0us/step
total 24
drwxr-xr-x 27 root root  4096 Mar 17 14:19 ImagesTrain
drwxr-xr-x  2 root root 20480 Mar 17 14:19 Test
2500


In [89]:
test_path = pathlib.Path('/root/.keras/datasets/Test')
images_test = os.listdir(test_path)
print(len(images_test))


500


In [90]:
for direct in train_path.iterdir():
  if direct.is_dir():
    direct.rename(train_path / direct.name.split(sep='_100')[0])

In [91]:
batch_size = 32
img_height = 224
img_width  = 224

In [93]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_ds_gen = ImageDataGenerator(vertical_flip    = True,
                                  horizontal_flip  = True,
                                  validation_split = 0.2)

In [94]:
train_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  subset='training',
                                                  class_mode='categorical')

Found 2000 images belonging to 25 classes.


In [95]:
valid_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  subset='validation',
                                                  class_mode='categorical')

Found 500 images belonging to 25 classes.


In [96]:
test_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  class_mode='categorical')

Found 2500 images belonging to 25 classes.


In [97]:
model.evaluate(valid_data_gen)

16/16 [==============================] - 81s 5s/step - loss: 0.3284 - f1_score: 0.9107


[0.3284420967102051, 0.9107266068458557]

Se carga manualmente y se descomprime.

# 4. Predicción

Definir función que haga las predicciones con el modelo sobre el set de test.

In [98]:
def make_predictions(model,
                     path_test ='Test-definitivo-de-Kaggle-03-2021',
                     name_file_submission = 'submission.csv'):

  images_test = os.listdir(path_test)
  samples_to_predict = []

  for img in images_test:
    path_img = os.path.join(path_test, img)
    img = image.load_img(path_img,
                        target_size=(img_width, img_height))
    x = image.img_to_array(img)
    samples_to_predict.append(x)

  samples_to_predict  = np.array(samples_to_predict)
  predictions = model_final.predict(samples_to_predict)
  classes = np.argmax(predictions, axis = 1)

  data = {'id.jpg': [img for img in images_test], 'label': classes}
  data = pd.DataFrame(data)
  data.to_csv(name_file_submission, index = False)

  return data



Hacer la predicción.


In [100]:
make_predictions(model, name_file_submission = 'resultados.csv' )

FileNotFoundError: ignored

# 5. Descarga de resultados

In [ ]:
from google.colab import files

files.download("resultados.csv")